In [1]:
!pip --version

# To install pytorch -------------------------------------------------------------------------------------
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# To install Tensorflow ----------------------------------------------------------------------------------
# Requires the latest pip
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

# To install Flax ----------------------------------------------------------------------------------------
!pip install flax

# or to install the latest version of Flax:
!pip install --upgrade git+https://github.com/google/flax.git

# Set up transformers
!pip install git+https://github.com/huggingface/transformers

# Check if installed
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('I love you'))"

!pip install transformers[torch]

!pip install accelerate -U

!pip install datasets

# !pip install git+https://github.com/lxuechen/private-transformers.git

!pip install opacus

!pip install evaluate

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/google/flax.git to /tmp/pip-req-build-5reforzn
  Running command git clone --filter=blob:none --quiet https://github.com/google/flax.git /tmp/pip-req-build-5reforzn
  Resolved https://github.com/google/flax.git to commit 2e51a4e5c69256969db5c3427b13b7ead269a573
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for flax: filename=flax-0.8.0-py3-none-any.whl size=634069 sha256=c13f11ed32017232e749f814b55dea4ff58b1bf229a9908afc34151b9cedf7d5
  Stored in directory: /tmp/p

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

metric = evaluate.load('accuracy')

In [3]:
from datasets.download.streaming_download_manager import xexists
# Libraries to import
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.losses import BinaryCrossentropy
from datasets import load_metric
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import pandas as pd
#from private_transformers import PrivacyEngine
import opacus
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Importing Dataset
dataset = load_dataset("sst2")

# Defining our model
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=2) # as output 0 or 1

model = model.to('cuda')

print(dataset)

print(dataset['train'])

# Tokenizing
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")

x_train = list(dataset["train"]["sentence"])
y_train = list(dataset["train"]["label"])

x_validation = list(dataset["validation"]["sentence"])
y_validation = list(dataset["validation"]["label"])

x_test = list(dataset["test"]["sentence"])
y_test = list(dataset["test"]["label"])

X_train_tokenized = tokenizer(x_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(x_validation, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(x_test, padding=True, truncation = True, max_length = 512)

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_validation)
test_dataset = Dataset(X_test_tokenized, y_test)

def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

  # Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=10,
    per_device_train_batch_size=200

)
privacy_engine = PrivacyEngine(
    model,
    batch_size=10,
    sample_size=50000,
    epochs=3,
    max_grad_norm=0.1,
    target_epsilon=3,
)
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
# model, optimizer, dataloader = privacy_engine.make_private(module=model, optimizer=optimizer, data_loader=dataloader , noise_multiplier=1.0, max_grad_norm=1.0 )
privacy_engine.attach(optimizer)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

#trainer.predict(test_dataset)


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

KeyboardInterrupt: ignored

Step 1: Data Loader : adds poisson sampling
Step 2: Data collate

In [2]:
from datasets.download.streaming_download_manager import xexists
from datasets import Dataset
# Libraries to import
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.losses import BinaryCrossentropy
from datasets import load_metric
import torch
import numpy as np
from transformers import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import pandas as pd
# from private_transformers import PrivacyEngine
import opacus
from opacus import PrivacyEngine
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import evaluate
from evaluate import evaluator

# Preprocessing
dataset = load_dataset("sst2")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=2) # as output 0 or 1
model = model.to('cuda')
# optimizer = AdamW(model.parameters(), lr=5e-5, eps = 1e-8)
metric = evaluate.load("accuracy")
task_evaluator = evaluator("text-classification")
# optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")

# Assuming the dataset has a 'train' split, modify this according to your dataset's splits
data_split = dataset['train']

# Convert the dataset split to a pandas DataFrame for easier splitting
df = data_split.to_pandas()

# Split the dataset into train and test sets using train_test_split from sklearn
# df1, df2 = train_test_split(df, test_size = 0.5, random_state = 42)
train_df, test_df = train_test_split(df , test_size=0.2, random_state=42)

# Convert the splits back to datasets
train_dataset = train_df.reset_index(drop=True)
test_dataset = test_df.reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

x_train = list(train_dataset["sentence"])
y_train = list(train_dataset["label"])

x_test = list(test_dataset["sentence"])
y_test = list(test_dataset["label"])

X_train_tokenized = tokenizer(x_train, padding=True, truncation=True, max_length=512)
# X_val_tokenized = tokenizer(x_validation, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(x_test, padding=True, truncation = True, max_length = 512)


class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to('cuda') for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx]).to('cuda')
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
# val_dataset = Dataset(X_val_tokenized, y_validation)
test_dataset = Dataset(X_test_tokenized, y_test)

train_dataloader = DataLoader(train_dataset, batch_size = 32, drop_last = True)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size = 32, drop_last = True)
# val_dataloader = DataLoader(val_dataset, batch_size = 500)

trainable_layers = [model.bert.encoder.layer[-1], model.bert.pooler, model.classifier]
total_params = 0
trainable_params = 0
l = []

for p in model.parameters():
        p.requires_grad = False
        total_params += p.numel()

for layer in trainable_layers:
    for p in layer.parameters():
        l.append(p)
        p.requires_grad = True
        trainable_params += p.numel()

optimizer = AdamW(l, lr=5e-5, eps = 1e-8)

print(f"Total parameters count: {total_params}") # ~108M
print(f"Trainable parameters count: {trainable_params}") # ~7M

def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):
    model.eval()

    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:
        #batch = tuple(t.to("cuda") for t in batch)

        with torch.no_grad():
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss, logits = outputs[:2]

            preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
            labels = batch['labels'].detach().cpu().numpy()

            loss_arr.append(loss.item())
            accuracy_arr.append(accuracy(preds, labels))

    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

model.train()
privacy_engine = PrivacyEngine()
model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    target_delta= 1 / len(train_dataloader),
    target_epsilon= 8,
    epochs= 3,
    max_grad_norm = 0.1,
)

# Training loop
optimizer.zero_grad()  # Explicitly zero the gradient buffers

for epoch in range(20):  # Number of epochs
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # print(f"input_ids size: {input_ids.size()}")
        # print(f"attention_mask size: {attention_mask.size()}")
        # print(f"labels size: {labels.size()}")

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        # print(outputs)
        loss = outputs[0]
        # print(loss)
        loss.backward()
        optimizer.step()

        logits = outputs[1]
        predictions = torch.argmax(logits, dim = -1)
        metric.add_batch(predictions = predictions, references = batch["labels"])

    eval_loss, eval_accuracy = evaluate(model)
    print("eval loss",eval_loss)
    print("accuracy: ",eval_accuracy)
    print("ends")
    #metric.compute()
    #print(metric)

    # Validation
    model.eval()
    for batch in test_dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            # Further validation steps if needed

    model.train()  # Set the model back to training mode




`AnnotionFormat` is deprecated and will be removed in v4.38. Please use `transformers.image_utils.AnnotationFormat` instead.


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((

Total parameters count: 4386178
Trainable parameters count: 215042


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1324: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "


eval loss 2.3090566270408175
accuracy:  0.5614583333333333
ends
eval loss 2.265826362797192
accuracy:  0.5614583333333333
ends
eval loss 1.990080697621618
accuracy:  0.5614583333333333
ends
eval loss 1.6733947890145437
accuracy:  0.562202380952381
ends
eval loss 1.1978786928313119
accuracy:  0.6253720238095238
ends
eval loss 1.1770558169200307
accuracy:  0.6792410714285714
ends
eval loss 1.3127943246137528
accuracy:  0.6982886904761905
ends
eval loss 1.3743680324582828
accuracy:  0.7069196428571428
ends
eval loss 1.3653986324157035
accuracy:  0.7157738095238095
ends
eval loss 1.3888878170933043
accuracy:  0.7192708333333333
ends
eval loss 1.3754098653083755
accuracy:  0.724032738095238
ends
eval loss 1.3700153582862444
accuracy:  0.7269345238095238
ends
eval loss 1.3707231853334676
accuracy:  0.7307291666666667
ends
eval loss 1.3411283164152077
accuracy:  0.7321428571428571
ends
eval loss 1.3428258469062193
accuracy:  0.7333333333333333
ends
eval loss 1.3278188792367775
accuracy:  0.73

In [ ]:
print(train_dataset[0])

In [ ]:
from torch.nn.utils import clip_grad_norm_

optimizer = torch.optim.SGD(lr=args.lr)

for batch in Dataloader(train_dataset, batch_size=32):
    for param in model.parameters():
        param.accumulated_grads = []

    # Run the microbatches
    for sample in batch:
        x, y = sample
        y_hat = model(x)
        loss = criterion(y_hat, y)
        loss.backward()

        # Clip each parameter's per-sample gradient
        for param in model.parameters():
            per_sample_grad = p.grad.detach().clone()
            clip_grad_norm_(per_sample_grad, max_norm=args.max_grad_norm)  # in-place
            param.accumulated_grads.append(per_sample_grad)

    # Aggregate back
    for param in model.parameters():
        param.grad = torch.stack(param.accumulated_grads, dim=0)

    # Now we are ready to update and add noise!
    for param in model.parameters():
        param = param - args.lr * param.grad
        param += torch.normal(mean=0, std=args.noise_multiplier * args.max_grad_norm)

        param.grad = 0  # Reset for next iteration

In [ ]:

def accuracy(preds, labels):
    return np.mean(preds == labels)

model.eval()  # Set the model to evaluation mode
train_preds = []
train_labels = []

for batch in train_dataloader:
    with torch.no_grad():
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)

        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

train_accuracy = accuracy(train_preds, train_labels)
print(f"Training Accuracy: {train_accuracy}")

# Calculate test accuracy
test_preds = []
test_labels = []

for batch in test_dataloader:
    with torch.no_grad():
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)

        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy(test_preds, test_labels)
print(f"Test Accuracy: {test_accuracy}")


Training Accuracy: 0.0
Test Accuracy: 0.0
